In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv("../data/7. train 이상치수정완료.csv",encoding='cp949',index_col=0)
test1 = pd.read_csv("../data/6. test1.csv",index_col=0)
test2 = pd.read_csv("../data/6. test2.csv",index_col=0)

In [2]:
from sklearn.preprocessing import StandardScaler

train_1 = train[train.columns[7:56]] # 필요한 데이터 셀렉

scaler = StandardScaler() # 스탠다드 스케일링
scaler.fit(train_1)
train_Std = scaler.transform(train_1)
X_train = pd.DataFrame(train_Std,columns = train_1.columns).drop('label',axis=1)
y_train = train_1[['label']]

t-검정을 위한 정규성검정(MDA)
- 등분산검정

In [3]:
# H가 1인 경우 이분산성 / H가 0인 경우 등분산
from scipy.stats import bartlett
def bartlett_test(col, p_value = 0.05, H = 1):
    list= []
    for i in col:
        T, p_val =bartlett(train_1[train_1['label']==1][i], train_1[train_1['label']==0][i]) 
        list.append([i, p_val])

    list = pd.DataFrame(list, columns = ['변수', 'p_value'])
    if H == 1:
        a = list[(list['p_value'] < p_value)][['변수', 'p_value']].sort_values('p_value')
        return a
    else:
        a = list[(list['p_value'] >= p_value)][['변수', 'p_value']].sort_values('p_value')
        return a

# 이분산성 변수
x_hetero = bartlett_test(train_1.columns, H = 1)
# 등분산성 변수
x_homo = bartlett_test(train_1.columns, H = 0)

c:\Users\dlsgm\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:2901: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\dlsgm\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:2901: RuntimeWarning: invalid value encountered in scalar subtract
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\dlsgm\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:2901: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\dlsgm\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:2901: RuntimeWarning: invalid value encountered in scalar subtract
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)


t-검정

In [4]:
# t-test

import scipy.stats as stats
def t_test(col, col_h0, col_h1, p_value = 0.05):
    list= []
    for i in col:
        if (col_h0['변수']==i).any():
            t_stat, p_val = stats.ttest_ind(train_1[train_1['label']==1][i], train_1[train_1['label']==0][i], equal_var=True) # 등분산성 : wald t-test
            list.append([i, p_val])
        elif (col_h1['변수']==i).any():
            t_stat, p_val = stats.ttest_ind(train_1[train_1['label']==1][i], train_1[train_1['label']==0][i], equal_var=False) # 이분산성 : welch’s t-test
            list.append([i, p_val])

    list = pd.DataFrame(list, columns = ['변수', 'p_value'])
    a = list[(list['p_value'] < p_value)][['변수', 'p_value']].sort_values('p_value').round(6)
    return a


# 2) t_test 결과 p_value < 0.05보다 작은 유의한 변수 가져오기
x_ttest = t_test(train_1.columns, x_homo, x_hetero, p_value=0.1)
print("유의한 피쳐 수 :", len(x_ttest))
x_ttest.sort_values(by="변수", ascending=True)

유의한 피쳐 수 : 33


,변수,p_value
13,1주당매출액,0.000000
14,1주당순이익,0.000000
43,기계투자효율,0.000176
16,당좌비율,0.028453
31,매입채무회전기간,0.037297
9,매출액순이익률,0.000001
40,부가가치,0.000032
44,부가가치율,0.000001
19,부채비율,0.000052
21,비유동비율,0.000118


In [5]:
# 유의한 피처들 중에서 p_value가 가장 작은 상위 10개의 '변수' 이름만 추출
top20_ttest= x_ttest.sort_values(by="p_value", ascending=True)['변수'].head(20).tolist()

# 결과 출력
print(len(top20_ttest))
print(top20_ttest)


20
['자기자본배율', '순이익증가율', '1주당매출액', '총자본순이익률', '1주당순이익', '총자본사업이익률', '자기자본순이익률', '자기자본증가율', '총자본투자효율', '총자본증가율', '매출액순이익률', '부가가치율', '순운전자본비율', '자기자본구성비율', '설비투자효율', '비유동자산증가율', '순운전자본회전률', '이자보상배율이자비용', '부가가치', '부채비율']


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

# 12개 피처로 조정 C 값 조정
lasso = SelectFromModel(LogisticRegression(C=0.2, penalty='l1',solver='liblinear'))
lasso.fit(X_train, y_train)
lasso_support = lasso.get_support()
lasso_feature = X_train.loc[:,lasso_support].columns.tolist()

print(len(lasso_feature))
print(lasso_feature)

17
['매출액증가율', '재고자산증가율', '자기자본증가율', '총자본증가율', '순이익증가율', '매출액총이익률', '자기자본순이익률', '순운전자본비율', '이자보상배율이자비용', '자기자본배율', '매출채권회전률', '1회전기간', '총자본투자효율', '설비투자효율', '기계투자효율', '부가가치율', '이윤분배율']


c:\Users\dlsgm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


공통피처

In [7]:
# lasso_feature에서 선택된 피처들과 top_10_feature_names에서 선택된 피처들의 교집합을 찾음
common_features = set(lasso_feature).intersection(top20_ttest)

# 공통 피처들 출력
print("공통 피처수:", len(common_features))
print("공통 피처들:", list(common_features))


공통 피처수: 10
공통 피처들: ['부가가치율', '자기자본순이익률', '총자본투자효율', '자기자본배율', '총자본증가율', '순이익증가율', '설비투자효율', '이자보상배율이자비용', '자기자본증가율', '순운전자본비율']


최종피처 10개

In [8]:
fea = ['총자본증가율','순이익증가율','자기자본순이익률','1주당매출액','1회전기간','매출채권회전률','자기자본배율','순운전자본비율','총자본투자효율','설비투자효율','label']

train_selected_feature= train[fea]
test1_selected_feature= test1[fea]
test2_selected_feature= test2[fea]

train_selected_feature.to_csv("../data/8-1 train(std).csv")  
test1_selected_feature.to_csv("../data/8-1 test1(std).csv") 
test2_selected_feature.to_csv("../data/8-1 test2(std).csv") 